 **Stored Procedures** using your simplified **`amazon`** database schema.

---

##  What is a Stored Procedure?

A **Stored Procedure** is a named block of SQL code that:

* Can perform multiple operations (SELECT, INSERT, UPDATE, etc.),
* Can accept **input**, **output**, or **INOUT** parameters,
* Is stored in the database and **invoked with `CALL`**,
* Doesn't have to return a value (unlike functions).

---

##  Syntax

```sql
DELIMITER $$

CREATE PROCEDURE procedure_name(IN param1 datatype, OUT param2 datatype)
BEGIN
    -- SQL statements
END $$

DELIMITER ;
```

 To **call** the procedure:

```sql
CALL procedure_name(arg1, @output_var);
SELECT @output_var;
```

---

##  Examples Using the `amazon` Database

---

###  1. Procedure to get total number of orders by a customer

```sql
DELIMITER $$

CREATE PROCEDURE get_order_count_by_customer(IN cust_id INT, OUT total_orders INT)
BEGIN
    SELECT COUNT(*) INTO total_orders
    FROM orders
    WHERE customer_id = cust_id;
END $$

DELIMITER ;
```

 **Usage:**

```sql
CALL get_order_count_by_customer(1, @orders);
SELECT @orders;
```

---

###  2. Procedure to get total order amount for a given order

```sql
DELIMITER $$

CREATE PROCEDURE get_order_total(IN o_id INT, OUT total_amount DECIMAL(10,2))
BEGIN
    SELECT SUM(p.price * oi.quantity) INTO total_amount
    FROM order_items oi
    JOIN products p ON oi.product_id = p.product_id
    WHERE oi.order_id = o_id;
END $$

DELIMITER ;
```

 **Usage:**

```sql
CALL get_order_total(101, @amount);
SELECT @amount;
```

---

###  3. Procedure to insert a new order for a customer (with one product for simplicity)

```sql
DELIMITER $$

CREATE PROCEDURE place_order(
    IN cust_id INT,
    IN prod_id INT,
    IN qty INT,
    IN ord_date DATE
)
BEGIN
    DECLARE new_order_id INT;

    -- Step 1: Insert into orders
    SELECT MAX(order_id) + 1 INTO new_order_id FROM orders;
    INSERT INTO orders(order_id, customer_id, order_date)
    VALUES (new_order_id, cust_id, ord_date);

    -- Step 2: Insert into order_items
    INSERT INTO order_items(order_id, product_id, quantity)
    VALUES (new_order_id, prod_id, qty);
END $$

DELIMITER ;
```

 **Usage:**

```sql
CALL place_order(4, 7, 2, '2023-04-01');
```

---


In [ ]:
# -- Simplified SQL Schema with 7 records in each table
# DROP DATABASE IF EXISTS `amazon`;
# CREATE DATABASE `amazon`;
# USE `amazon`;
# -- Order Items Table (Many-to-Many Relationship)

# -- Drop existing tables to avoid conflicts
# DROP TABLE IF EXISTS order_items;
# DROP TABLE IF EXISTS orders;
# DROP TABLE IF EXISTS products;
# DROP TABLE IF EXISTS customers;

# -- Customers Table
# CREATE TABLE customers (
#   customer_id INT PRIMARY KEY,
#   name VARCHAR(50)
# );

# INSERT INTO customers VALUES
# (1, 'Alice'),
# (2, 'Bob'),
# (3, 'Charlie'),
# (4, 'Diana'),
# (5, 'Ethan'),
# (6, 'Fiona'),
# (7, 'George');

# -- Orders Table
# CREATE TABLE orders (
#   order_id INT PRIMARY KEY,
#   customer_id INT,
#   order_date DATE,
#   FOREIGN KEY (customer_id) REFERENCES customers(customer_id)
# );

# INSERT INTO orders VALUES
# (101, 1, '2023-01-01'),
# (102, 1, '2023-01-10'),
# (103, 2, '2023-02-05'),
# (104, 3, '2023-02-20'),
# (105, 5, '2023-03-01'),
# (106, 6, '2023-03-15'),
# (107, 7, '2023-03-25');

# -- Products Table
# CREATE TABLE products (
#   product_id INT PRIMARY KEY,
#   name VARCHAR(50),
#   price DECIMAL(5,2)
# );

# INSERT INTO products VALUES
# (1, 'Pen', 1.50),
# (2, 'Notebook', 2.75),
# (3, 'Backpack', 15.00),
# (4, 'Pencil', 0.99),
# (5, 'Eraser', 0.50),
# (6, 'Marker', 1.80),
# (7, 'Stapler', 3.99);



# CREATE TABLE order_items (
#   order_id INT,
#   product_id INT,
#   quantity INT,
#   PRIMARY KEY (order_id, product_id),
#   FOREIGN KEY (order_id) REFERENCES orders(order_id),
#   FOREIGN KEY (product_id) REFERENCES products(product_id)
# );

# INSERT INTO order_items VALUES
# (101, 1, 2),
# (101, 2, 1),
# (102, 3, 1),
# (103, 1, 5),
# (104, 4, 3),
# (105, 5, 2),
# (106, 6, 1);
